In [19]:
import pandas as pd

In [20]:
df_candidates = pd.read_csv("D:/学习/LUNA16/CSVFILES/CSVFILES/candidates.csv")
df_candidates.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


In [21]:
df_candidates[df_candidates["class"]==1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1351 entries, 13 to 550906
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   seriesuid  1351 non-null   object 
 1   coordX     1351 non-null   float64
 2   coordY     1351 non-null   float64
 3   coordZ     1351 non-null   float64
 4   class      1351 non-null   int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 63.3+ KB


In [22]:
df_annonation = pd.read_csv("D:/学习/LUNA16/CSVFILES/CSVFILES/annotations.csv")
df_annonation.head(5)

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [23]:
from collections import namedtuple
import functools,os,glob

In [24]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple','isNodule_bool, diameter_mm, series_uid, center_xyz')


In [32]:
@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool = True):
    df_candidates = pd.read_csv("D:/学习/LUNA16/CSVFILES/CSVFILES/candidates.csv")
    df_annonation = pd.read_csv("D:/学习/LUNA16/CSVFILES/CSVFILES/annotations.csv")
    mhd_list = glob.glob("D:/学习/LUNA16/subset*/subset*/*.mhd")
    presentOnDisk_Set = {os.path.split(p)[-1][:-4] for p in mhd_list}
    diameter_dict = {}
    
    for index,row in df_annonation.iterrows():
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationCenter_mm = float(row[4])
        diameter_dict.setdefault(series_uid,[]).append((annotationCenter_xyz,annotationCenter_mm))
    
    candidataeInfo_list = []
    for index,row in df_candidates.iterrows():
        series_uid = row[0]
        if series_uid not in presentOnDisk_Set and requireOnDisk_bool:
            continue
        isNodule_bool = bool(int(row[4]))
        candidateCenter_xyz = tuple([float(x) for x in row[1:4]])
        candidataDiameter_mm = 0.0
        for annotationCenter_tup in diameter_dict.get(series_uid,[]):
            annotationCenter_xyz,annotationCenter_mm = annotationCenter_tup
            for i in range(3):
                delta_mm = abs(annotationCenter_xyz[i] - candidateCenter_xyz[i])
                if delta_mm>annotationCenter_mm/4:
                    break
            else:
                candidataDiameter_mm = annotationCenter_mm
                break
        candidataeInfo_list.append(CandidateInfoTuple(
            isNodule_bool,
            candidataDiameter_mm,
            series_uid,
            candidateCenter_xyz
        ))
        
        

In [39]:
list_ = [(1,2),(1,1),(1,3)]
print(list_.sort)

1
==
1
==
1
==
1
==
